In [118]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC

Reading files: 

In [119]:
train_data = sp.genfromtxt("data/train_data.txt", delimiter="\t")

test_data = sp.genfromtxt("data/test_data.txt", delimiter="\t")

val_data =  sp.genfromtxt("data/val_data.txt", delimiter="\t")

Dimensions:

In [120]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(858L, 25L)
(286L, 25L)
(286L, 25L)


In [121]:
train_data = train_data[~np.isnan(train_data).any(axis=1)]
test_data = test_data[~np.isnan(test_data).any(axis=1)]
val_data = val_data[~np.isnan(val_data).any(axis=1)]
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(857L, 25L)
(286L, 25L)
(286L, 25L)


Brisanje proba: 

In [122]:
train_data[:1]

array([[  4.79000000e+02,   8.10000000e+01,   2.29000000e+01,
          4.08000000e+00,   7.65000000e+00,   5.34000000e-01,
          1.00000000e-02,   4.00000000e-02,   1.76000000e-01,
          2.72000000e+00,   3.45000000e+00,   7.88000000e-01,
          1.98000000e+00,   3.03000000e+00,   5.01000000e+00,
          6.80000000e-01,   4.30000000e-01,   4.10000000e-01,
          2.39000000e+00,   1.22000000e+00,   1.08900000e+01,
          8.00000000e+00,   3.40000000e+01,   1.12000000e+02,
          2.06000000e+02]])

In [123]:
np.delete(train_data[:1], 1)

array([  4.79000000e+02,   2.29000000e+01,   4.08000000e+00,
         7.65000000e+00,   5.34000000e-01,   1.00000000e-02,
         4.00000000e-02,   1.76000000e-01,   2.72000000e+00,
         3.45000000e+00,   7.88000000e-01,   1.98000000e+00,
         3.03000000e+00,   5.01000000e+00,   6.80000000e-01,
         4.30000000e-01,   4.10000000e-01,   2.39000000e+00,
         1.22000000e+00,   1.08900000e+01,   8.00000000e+00,
         3.40000000e+01,   1.12000000e+02,   2.06000000e+02])

# Features

In [124]:
def np_featuriser(train, test, val,feature_list):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val


In [125]:
train_all, test_all, val_all = np_featuriser(train_data, test_data, val_data,22)

# Labels

In [126]:
train_labels = train_data[:,22]
test_labels = test_data[:,22]
val_labels = val_data[:,22]
print(train_labels.shape)
print(test_labels.shape)
print(val_labels.shape)

(857L,)
(286L,)
(286L,)


In [127]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_data, test_data, val_data, feature_list )

In [128]:
#train_features = np.delete(train_data, 22, 1)
#test_features = np.delete(test_data, 22, 1)
#val_features = np.delete(val_data,22,1)
                          
#train_features = np.delete(train_features, 21, 1)
#test_features = np.delete(test_features, 21, 1)
#val_features = np.delete(val_features,21,1)
                          
#print(train_features.shape)
#print(test_features.shape)
#print(val_features.shape)

columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 


# Univariate feature selection

In [167]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.19:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove

[  11.11071711   64.4679563     1.82896326   73.92406058   27.41925933
   42.04831989   75.91498997   28.98983588   59.2040295    12.51662655
   10.79262582  218.66202788  287.90107153]
[ 0.03859214  0.22392399  0.00635275  0.25676897  0.09523848  0.14605128
  0.26368429  0.10069374  0.20564018  0.04347544  0.03748727  0.75950404
  1.        ]
Most valuable features: 
1 0.223923988737 GS
3 0.256768966462 FGM
6 0.263684291166 3PM
8 0.205640184623 3P%
11 0.759504043217 FT%
12 1.0 OFF
[0, 2, 4, 5, 7, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [130]:
#feature_list = [0, 1, 2, 3, 4, 5, 9, 10, 18, 19, 20, 21, 22]
#train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_data, test_data, val_data, feature_list )

In [131]:
feature_list = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]

train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_data, test_data, val_data, feature_list )

# Feature selection with VarianceThreshold 

In [182]:
from sklearn.feature_selection import VarianceThreshold



In [185]:
def vt_fsel(train):
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(train)
    vt_list = sel.get_support()
    l_vt = []
    j = 0
    for i in vt_list:
        if i == False:
            l_vt.append(j)
            print "%s. featue name: %s" %(j, columns.keys()[columns.values().index(j)])
        j = j+1
    return l_vt    

In [187]:
list_vt = vt_fsel(train_data)

5. featue name: FG%
8. featue name: 3P%
11. featue name: FT%
16. featue name: STL


In [137]:
#train_features1 = sel.fit_transform(train_features)
#val_features1 = np.delete(val_features, 11, 1)
#train_features1.shape
#train_labels.shape

In [138]:
print "Training features dimensions: "  
print train_features_nb.shape
print "Test features dimensions: " 
print test_features_nb.shape
print "Validation features dimensions: "
print val_features_nb.shape

Training features dimensions: 
(857L, 13L)
Test features dimensions: 
(286L, 13L)
Validation features dimensions: 
(286L, 13L)


# Naive Bayes

In [139]:
clf = GaussianNB()

In [140]:
clf.fit(train_features_nb, train_labels)

GaussianNB()

In [141]:
pred = clf.predict(test_features_nb)

# Accuracy

In [142]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred, test_labels)
accuracy
#51% sa Variance Threshold
#53% with Univariate feature selection "SelectPercentile, f_classif"
#60% domensko znanjeI g

0.59790209790209792

# Custom Accuracy function

In [197]:
def custom_acc(pred_labels, test_labels):
    hit = 0
    num = 0
    j = 0
    for i in pred_labels:
        a = i
        a = str(int(a))
        b = test_labels[j]
        b = str(int(b))
        if a == b:
            hit = hit+1
        elif len(a) == 1 and len(b) == 2:
            if a == b[0] or a == b[1]:
                hit = hit+1
        elif len(a) == 2 and len(b) == 1:
            if a[0] == b or a[1] == b:
                hit = hit+1
        num = num+1
        j = j+1
    #print num
    #print hit
    return float(hit)/num


# Custom Accuracy NB

In [144]:
custom_acc(pred, test_labels)

286
241


0.8426573426573427

# Confusion matrix NB

In [145]:
from sklearn.metrics import confusion_matrix 

In [146]:
confm_nb = confusion_matrix(test_labels, pred)
confm_nb
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[12,  0,  0,  0,  0,  2,  0,  0,  0],
       [ 2,  7,  3,  0,  0, 22,  2,  0,  0],
       [ 0,  0,  9,  0,  0,  1,  7,  3,  0],
       [ 0,  0,  0, 15, 13,  0,  0,  9,  2],
       [ 0,  0,  0,  1, 38,  0,  0,  0,  8],
       [ 2,  1,  0,  0,  0, 54,  2,  0,  0],
       [ 0,  0,  2,  0,  0,  1,  2,  0,  0],
       [ 0,  0,  1,  0,  8,  6,  7, 32,  0],
       [ 0,  0,  0,  1,  8,  0,  0,  1,  2]])

# SVM

In [147]:
from sklearn.svm import SVC

In [148]:
clfSVM= SVC()

In [149]:
clfSVM.set_params(kernel = 'linear')

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [150]:
clfSVM.fit(train_features_nb, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [151]:
predSVM = clfSVM.predict(test_features_nb)

# Accuracy SVM

In [152]:
accuracySVM = accuracy_score(predSVM, test_labels)
accuracySVM
# < 50% Variance threshold 
# 53% Univariate feature selection "SelectPercentile, f_classif"
# 57% Domain knowledge

0.57692307692307687

# Custom Accuracy SVM

In [153]:
custom_acc(predSVM, test_labels)

286
249


0.8706293706293706

# Confusion matrix SVM

In [154]:
confm_svm = confusion_matrix(test_labels, predSVM)
confm_svm
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[ 8,  0,  0,  0,  0,  6,  0,  0,  0],
       [ 0,  8,  3,  0,  0, 25,  0,  0,  0],
       [ 0,  1, 11,  0,  0,  1,  2,  5,  0],
       [ 0,  0,  1, 13,  6,  0,  0, 15,  4],
       [ 0,  0,  0,  2, 42,  0,  0,  2,  1],
       [ 5,  3,  1,  0,  0, 47,  3,  0,  0],
       [ 0,  2,  1,  0,  0,  2,  0,  0,  0],
       [ 0,  1,  7,  1,  1,  1,  7, 34,  2],
       [ 0,  0,  0,  2,  5,  0,  0,  3,  2]])

# Logistic Regression

In [155]:
from sklearn import linear_model

In [156]:
logreg = linear_model.LogisticRegression(C=1e5)

In [157]:
logreg.fit(train_features_nb, train_labels)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [158]:
predLR = logreg.predict(test_features_nb)

# Logistic Regression Accuracy

In [159]:
accLR = accuracy_score(predLR, test_labels)
accLR
#50% Variance Threshold
#55% Univariate
#57% domain knowledge

0.56993006993006989

# Custom Accuracy SVM

In [160]:
custom_acc(predLR, test_labels)

286
240


0.8391608391608392

# Confusion matrix LR

In [161]:
confm_lr = confusion_matrix(test_labels, predLR)
confm_lr
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[ 7,  0,  0,  0,  0,  7,  0,  0,  0],
       [ 0,  5,  1,  0,  0, 25,  1,  4,  0],
       [ 1,  3,  1,  0,  0,  3,  0, 12,  0],
       [ 0,  0,  0, 12,  9,  0,  0, 17,  1],
       [ 0,  0,  0,  1, 43,  0,  0,  2,  1],
       [ 2,  1,  0,  0,  0, 52,  0,  4,  0],
       [ 0,  1,  0,  0,  0,  2,  0,  2,  0],
       [ 0,  0,  0,  3,  2,  6,  1, 42,  0],
       [ 0,  0,  0,  1,  5,  0,  0,  5,  1]])

In [192]:
list_none = [22]
list_domain = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
list_uni = [0, 2, 4, 5, 7, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [199]:
def print_metrics(name, accuracy, c_acc, conf_matrix):
    print "Feature selection: %s\n" %name
    print "Accuracy score: %s\n" %accuracy
    print "Custom Accuracy score: %s\n" %c_acc
    print "Confusion matrix:"
    print "\n%s" %conf_matrix
    print"\n"
    
def clf(clf, tr, tr_labels, val, val_labels):
        clf.fit(tr, tr_labels)
        
        pred = clf.predict(val)
        
        acc = metrics.accuracy_score(val_labels,pred)
        c_acc = custom_acc(pred, val_labels)
        conf = metrics.confusion_matrix(val_labels, pred)
        
        return acc, c_acc, conf

In [200]:
def clf_all(CLF, l_none, l_domain, l_uni, l_vt, train_all, test_all, val_all):
    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
    #print tr_none.shape
    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
    
    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
    
    tr_vt, ts_vt, val_vt = np_featuriser(train_all, test_all, val_all, l_vt)
    
    #clfnb = GaussianNB()
    #print "Naive Bayes\n"
    
    acc_none, c_acc, conf_none = clf(CLF, tr_none, train_labels, val_none, val_labels)
    print_metrics("None", acc_none, c_acc, conf_none)
    
    acc_domain, c_acc, conf_domain = clf(CLF, tr_domain, train_labels, val_domain, val_labels)
    print_metrics("Domain knowledge", acc_domain, c_acc, conf_domain)

    acc_uni,c_acc, conf_uni = clf(CLF, tr_uni, train_labels, val_uni, val_labels)
    print_metrics("Univariate", acc_uni, c_acc, conf_uni)
    
    acc_vt, c_acc, conf_vt = clf(CLF, tr_vt, train_labels, val_vt, val_labels)
    print_metrics("Variance Threshold", acc_vt, c_acc, conf_vt)

In [201]:
clfnb = GaussianNB()
clf_all(clfnb, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

Feature selection: None

Accuracy score: 0.527972027972

Custom Accuracy score: 0.741258741259

Confusion matrix:

[[10  0  0  0  0  4  0  0  0]
 [ 2 11  3  0  2 15  3  0  0]
 [ 0  1 10  0  0  1  4  4  0]
 [ 0  0  0 13 18  0  0  4  4]
 [ 0  0  0  3 36  0  0  0  8]
 [ 2  0  0  0  1 53  3  0  0]
 [ 0  1  2  0  0  1  1  0  0]
 [ 0  0  1  1 27  3  7 15  0]
 [ 0  0  0  0  8  0  0  2  2]]


Feature selection: Domain knowledge

Accuracy score: 0.583916083916

Custom Accuracy score: 0.832167832168

Confusion matrix:

[[10  0  0  0  0  4  0  0  0]
 [ 1  8  3  0  0 23  1  0  0]
 [ 0  0 10  0  0  2  5  3  0]
 [ 0  0  0 14 11  0  0 11  3]
 [ 0  0  0  1 38  0  0  0  8]
 [ 2  1  0  0  0 55  1  0  0]
 [ 0  0  2  0  0  2  1  0  0]
 [ 0  0  1  1 12  5  6 29  0]
 [ 0  0  0  0  6  0  0  4  2]]


Feature selection: Univariate

Accuracy score: 0.381118881119

Custom Accuracy score: 0.524475524476

Confusion matrix:

[[ 0  7  0  0  2  5  0  0  0]
 [ 0 10  4  0  2 19  0  1  0]
 [ 0  4  5  2  3  6  0  0  0]
 

In [202]:
svm = SVC()
svm.set_params(kernel='linear')
clf_all(svm, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

Feature selection: None

Accuracy score: 0.618881118881

Custom Accuracy score: 0.902097902098

Confusion matrix:

[[ 7  0  0  0  0  7  0  0  0]
 [ 0 14  0  0  0 21  1  0  0]
 [ 0  2  9  0  0  1  3  5  0]
 [ 0  0  0 17  5  0  0 14  3]
 [ 0  0  0  1 42  0  0  3  1]
 [ 3  3  2  0  0 51  0  0  0]
 [ 0  2  1  0  0  2  0  0  0]
 [ 0  0  6  4  3  2  2 37  0]
 [ 0  0  0  2  5  0  0  5  0]]


Feature selection: Domain knowledge

Accuracy score: 0.583916083916

Custom Accuracy score: 0.877622377622

Confusion matrix:

[[ 8  0  0  0  0  6  0  0  0]
 [ 0  8  1  0  0 26  1  0  0]
 [ 0  1  9  0  0  2  4  4  0]
 [ 0  0  1 11  7  0  0 16  4]
 [ 0  0  0  3 42  0  0  1  1]
 [ 2  2  0  0  0 53  1  1  0]
 [ 0  2  1  0  0  2  0  0  0]
 [ 0  0  3  4  2  2  6 35  2]
 [ 0  0  0  3  5  0  0  3  1]]


Feature selection: Univariate

Accuracy score: 0.405594405594

Custom Accuracy score: 0.629370629371

Confusion matrix:

[[ 0  0  1  0  0 13  0  0  0]
 [ 0  1  2  0  0 28  0  5  0]
 [ 0  0  2  1  2 10  0  5  0]
 

In [203]:
logreg = linear_model.LogisticRegression(C=1e5)
clf_all(logreg, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

Feature selection: None

Accuracy score: 0.548951048951

Custom Accuracy score: 0.79020979021

Confusion matrix:

[[ 6  0  0  0  0  8  0  0  0]
 [ 1  8  3  0  0 20  0  4  0]
 [ 0  2  4  4  0  2  0  8  0]
 [ 0  0  0 14 12  0  0 12  1]
 [ 0  0  0  1 44  0  0  1  1]
 [ 8  0  0  0  0 43  1  7  0]
 [ 0  1  0  0  0  2  0  2  0]
 [ 0  0  0  4  5  6  1 38  0]
 [ 0  0  0  1  5  0  0  6  0]]


Feature selection: Domain knowledge

Accuracy score: 0.555944055944

Custom Accuracy score: 0.818181818182

Confusion matrix:

[[ 5  0  0  0  0  9  0  0  0]
 [ 0  5  1  0  0 25  1  4  0]
 [ 0  2  1  2  0  3  0 12  0]
 [ 0  0  0 15 12  0  0 12  0]
 [ 0  0  0  0 43  0  0  3  1]
 [ 3  1  0  0  0 51  0  4  0]
 [ 0  0  0  0  0  2  0  3  0]
 [ 0  0  0  3  5  6  1 39  0]
 [ 0  0  0  2  6  0  0  4  0]]


Feature selection: Univariate

Accuracy score: 0.409090909091

Custom Accuracy score: 0.622377622378

Confusion matrix:

[[ 1  0  1  0  0 12  0  0  0]
 [ 1  1  1  0  0 25  0  8  0]
 [ 0  0  3  1  2 10  0  4  0]
 [